# Settings

In [1]:
%env TF_KERAS = 1
import os
sep_local = os.path.sep

import sys
sys.path.append('..'+sep_local+'..')
print(sep_local)

env: TF_KERAS=1
\


In [2]:
import tensorflow as tf
print(tf.__version__)

2.1.0


# Dataset loading

In [3]:
dataset_name='pokemon'

In [4]:
IMG_DIR = 'C:\\Users\\Khalid\\Documents\projects\\pokemon\DS06\\'
VAL_PCT = 10
VAL_FORMAT = 'png'

In [5]:
os.chdir('..'+sep_local+'..'+sep_local+'..'+sep_local+'..'+sep_local+'..')

In [6]:
from training.generators.file_image_generator import create_image_lists, get_generators

Using TensorFlow backend.


In [7]:
imgs_list = create_image_lists(
    image_dir=IMG_DIR, 
    validation_pct=VAL_PCT, 
    valid_imgae_formats=VAL_FORMAT
)

  DEBUG    | Looking for images in 'all'


  INFO     | 809 file found


In [8]:
inputs_shape= IMG_SIZE=(200, 200, 3)
BATCH_SIZE = 32
latent_dim = 32
intermediate_dim = 16

In [9]:
train_gen, test_gen = get_generators(
    images_list=imgs_list, 
    image_dir=IMG_DIR, 
    image_size=IMG_SIZE, 
    batch_size=BATCH_SIZE, 
    class_mode=None
)

  INFO     | Found 738 training files


  INFO     | Found 71 validation files


In [10]:
import tensorflow as tf

In [11]:
train_ds = tf.data.Dataset.from_generator(
    lambda: train_gen, 
    output_types=tf.float32 ,
    output_shapes=tf.TensorShape((BATCH_SIZE, ) + IMG_SIZE)
)

test_ds = tf.data.Dataset.from_generator(
    lambda: test_gen, 
    output_types=tf.float32 ,
    output_shapes=tf.TensorShape((BATCH_SIZE, ) + IMG_SIZE)
)


In [12]:
_instance_scale=1.0
for data in train_ds:
    _instance_scale = float(data[0].numpy().max())
    break

In [13]:
_instance_scale

1.0

In [14]:
import numpy as np
from collections.abc import Iterable

In [15]:
if isinstance(inputs_shape, Iterable):
    _outputs_shape = np.prod(inputs_shape)

In [16]:
_outputs_shape

120000

# Model's Layers definition

In [17]:
enc_lays = [tf.keras.layers.Dense(units=intermediate_dim, activation='relu'),
            tf.keras.layers.Flatten(),
            tf.keras.layers.Dense(units=latent_dim)]

dec_lays = [tf.keras.layers.Dense(units=latent_dim, activation='relu'),
            tf.keras.layers.Dense(units=intermediate_dim, activation='relu'),
            tf.keras.layers.Dense(units=_outputs_shape),
            tf.keras.layers.Reshape(inputs_shape)]

# Model definition

In [19]:
model_name = dataset_name+'ILAAE'
recoding_dir='recoding'+sep_local+model_name

In [20]:
from training.inputs_latent_adversarial.autoencoders.AAE import AAE as AE

In [21]:
inputs_shape=IMG_SIZE

In [22]:
variables_params = \
[
    {
        'name': 'inference', 
        'inputs_shape':inputs_shape,
        'outputs_shape':latent_dim,
        'layers': enc_lays
    }

    ,
    
        {
        'name': 'generative', 
        'inputs_shape':latent_dim,
        'outputs_shape':inputs_shape,
        'layers':dec_lays
    }
]

In [23]:
from utils.data_and_files.file_utils import create_if_not_exist

In [24]:
_restore = os.path.join(recoding_dir, 'var_save_dir')

In [25]:
create_if_not_exist(_restore)
_restore

'..\\..\\recoding\\pokemonGAAE\\var_save_dir'

In [26]:
#to restore trained model, set filepath=_restore

In [27]:
ae = AE( 
    name=model_name,
    inputs_shape=inputs_shape,
    outputs_shape=inputs_shape,
    latent_dim=latent_dim,
    batch_size=BATCH_SIZE,
    variables_params=variables_params, 
    filepath=None
    )

Model: "inference"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inference_inputs (InputLayer [(None, 200, 200, 3)]     0         
_________________________________________________________________
dense (Dense)                (None, 200, 200, 32)      128       
_________________________________________________________________
dense_1 (Dense)              (None, 200, 200, 32)      1056      
_________________________________________________________________
flatten (Flatten)            (None, 1280000)           0         
_________________________________________________________________
dense_2 (Dense)              (None, 32)                40960032  
_________________________________________________________________
batch_normalization (BatchNo (None, 32)                128       
_________________________________________________________________
dropout (Dropout)            (None, 32)                0 

  WARNING  | None


Model: "generative"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
generative_inputs (InputLaye [(None, 32)]              0         
_________________________________________________________________
dense_3 (Dense)              (None, 32)                1056      
_________________________________________________________________
dense_4 (Dense)              (None, 32)                1056      
_________________________________________________________________
dense_5 (Dense)              (None, 120000)            3960000   
_________________________________________________________________
reshape (Reshape)            (None, 200, 200, 3)       0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 200, 200, 3)       12        
_________________________________________________________________
dropout_1 (Dropout)          (None, 200, 200, 3)       0

  WARNING  | None


In [28]:
#ae.compile(metrics=None)
ae.compile()

Model: "pokemonGAAE"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inference_inputs (InputLayer [(None, 200, 200, 3)]     0         
_________________________________________________________________
inference (Model)            (None, 32)                40961344  
_________________________________________________________________
generative (Model)           (None, 200, 200, 3)       3962124   
_________________________________________________________________
tf_op_layer_x_logits (Tensor [(None, 200, 200, 3)]     0         
Total params: 44,923,468
Trainable params: 44,923,398
Non-trainable params: 70
_________________________________________________________________
None


# Callbacks

In [29]:
from training.callbacks.progress_bar import NotebookPrograssBar
from training.callbacks.sample_generation import SampleGeneration
from training.callbacks.save_model import ModelSaver

In [30]:
progbar = NotebookPrograssBar(leave_outer=False)

In [31]:
es = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss', 
    min_delta=1e-12, 
    patience=5, 
    verbose=1, 
    restore_best_weights=True
)

In [32]:
ms = ModelSaver(filepath=_restore,save_freq=5)

In [33]:
csv_dir = os.path.join(recoding_dir, 'csv_dir')
create_if_not_exist(csv_dir)
csv_dir = os.path.join(csv_dir, ae.name+'.csv')
csv_log = tf.keras.callbacks.CSVLogger(csv_dir, append=True)
csv_dir

'..\\..\\recoding\\pokemonGAAE\\csv_dir\\pokemonGAAE.csv'

In [34]:
image_gen_dir = os.path.join(recoding_dir, 'image_gen_dir')
create_if_not_exist(image_gen_dir)

In [35]:
sg = SampleGeneration(latent_shape=latent_dim, filepath=image_gen_dir, gen_freq=5, save_img=True, gray_plot=False)

In [36]:
import numpy as np

# Model Training

In [39]:
ae.fit(
    x=train_ds,
    input_kw=None,
    steps_per_epoch=10,
    epochs=10,#int(1e6), 
    verbose=1,
    #callbacks=[progbar, es, ms, csv_log, sg],
    workers=-1,
    use_multiprocessing=True,
    #validation_data=test_ds,
    #validation_steps=100
)


training traditional basicAE
Train for 10 steps
Epoch 1/10
10/10 [==============================] - ETA: 3s - loss: 77772.8516 - psnr: 8.7412 - ssmi: 0.0080 - sharp_diff: 4.63 - ETA: 2s - loss: 76812.4336 - psnr: 8.7594 - ssmi: 0.0080 - sharp_diff: 4.63 - ETA: 2s - loss: 75489.9714 - psnr: 8.8345 - ssmi: 0.0081 - sharp_diff: 4.64 - ETA: 1s - loss: 76998.7812 - psnr: 8.7535 - ssmi: 0.0081 - sharp_diff: 4.64 - ETA: 1s - loss: 75278.0437 - psnr: 8.8666 - ssmi: 0.0082 - sharp_diff: 4.64 - ETA: 1s - loss: 76425.2031 - psnr: 8.8125 - ssmi: 0.0082 - sharp_diff: 4.64 - ETA: 0s - loss: 77063.2500 - psnr: 8.7832 - ssmi: 0.0082 - sharp_diff: 4.64 - ETA: 0s - loss: 77476.7266 - psnr: 8.7589 - ssmi: 0.0082 - sharp_diff: 4.64 - ETA: 0s - loss: 77100.9071 - psnr: 8.7798 - ssmi: 0.0081 - sharp_diff: 4.64 - 3s 277ms/step - loss: 77467.0961 - psnr: 8.7642 - ssmi: 0.0082 - sharp_diff: 4.6469
Epoch 2/10
10/10 [==============================] - ETA: 2s - loss: 78823.3281 - psnr: 8.6847 - ssmi: 0.0086 - sh

ValueError: Unknown entries in loss dictionary: ['latent_real_discriminator_outputs']. Only expected following keys: ['logits_real_discriminator_outputs']